In [21]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim

#import matplotlib.pyplot as plt

In [2]:
# Parametry zestawu danych


data_mean = 4
data_stddev = 1.5

(name, preprocess, d_input_func) =  ("4 momenty centralne", lambda data: get_moments(data), lambda x :4)

In [6]:
print("Dane {}".format(name))

def get_distribution_samples(mu, sigma):
    return lambda n: torch.Tensor(np.random.normal(mu, sigma, (1, n)))


def get_generator_input_sampler():
    return lambda m, n : torch.rand(m, n)

Dane 4 momenty centralne


In [11]:
### Modele : Generator oraz DYskryminator


class Generator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, f):
        super(Generator, self).__init__()
        self.map1 = nn.Linear(input_size, hidden_size)
        self.map2 = nn.Linear(hidden_size, hidden_size)
        self.map3 = nn.Linear(hidden_size, output_size)
        self.f = f
        
        
    # Forward pass
    
    def forward(self, x):
        x = self.map1(x)
        x = self.f(x)
        x = self.map2(x)
        x = self.f(x)
        x = self.map3(x)
        return x
    
class Discriminator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, f):
        super(Discriminator, self).__init__()
        self.map1 = nn.Linear(input_size, hidden_size)
        self.map2 = nn.Linear(hidden_size, hidden_size)
        self.map3 = nn.Linear(hidden_size, output_size)
        self.f = f
        
    def forward(self, x):
        x = self.f(self.map1(x))
        x = self.f(self.map2(x))
        return self.f(self.map3(x))

def extract(v):
    return v.data.storage().tolist()

def stats(d):
    return [np.mean(d), np.std(d)]


def get_moments(d):
    mean = torch.mean(d)
    diffs = d - mean
    var = torch.mean(torch.pow(diffs, 2.0))
    std = torch.pow(var, 0.5)
    zscores = diffs /  std
    skews = torch.mean(torch.pow(zscores, 3.0 ))
    kurtoses = torch.mean(torch.pow(zscores, 4.0)) - 3.0
    final = torch.cat((mean.reshape(1,), std.reshape(1,), skews.reshape(1,), kurtoses.reshape(1,)))
    return final

def decorate_with_diffs(data, exponent, remove_raw_data=False):
    mean = torch.mean(data.data, 1, keepdim=True)
    mean_broadcast = torch.mul(torch.ones(data.size()), mean.tolist()[0][0])
    diffs = torch.pow(data - Variable(mean_broadcast), exponent)
    if remove_raw_data:
        return torch.cat([diffs],1)
    else:
        return torch.cat([data, diffs],1)
    

In [22]:
def train():
    
    g_input_size = 1
    g_hidden_size = 5
    g_output_size = 1
    d_input_size = 500
    d_hidden_size = 10
    d_output_size = 1
    minibatch_size = d_input_size
    
    
    d_learning_rate = 1e-3
    g_learning_rate = 1e-3
    sgd_momentum = 0.9
    
    
    num_epochs = 5000
    print_interval = 100
    d_steps = 20
    g_steps = 20
    
    
    dfe, dre, ge = 0, 0, 0
    d_real_data, d_fake_data, g_fake_data = None, None, None
    discriminator_activation_function = torch.sigmoid
    generator_activation_function = torch.tanh
    
    
    d_sampler = get_distribution_samples(data_mean, data_stddev)
    #gi_samples = get_generator_input_sampler()
    gi_sampler = get_generator_input_sampler()
    G = Generator(input_size=g_input_size,
                hidden_size=g_hidden_size,
                output_size=g_output_size,
                f=generator_activation_function)
    D = Discriminator(input_size=d_input_func(d_input_size),
                hidden_size=d_hidden_size,
                output_size=d_output_size,
                f=discriminator_activation_function)
    criterion = nn.BCELoss()  # Binary cross entropy: http://pytorch.org/docs/nn.html#bceloss
    d_optimizer = optim.SGD(D.parameters(), lr=d_learning_rate, momentum=sgd_momentum)
    g_optimizer = optim.SGD(G.parameters(), lr=g_learning_rate, momentum=sgd_momentum)
    
    for epoch in range(num_epochs):
        for d_index in range(d_steps):
            D.zero_grad()
            
            # 1A : Train D on  real
            d_real_data = Variable(d_sampler(d_input_size))
            d_real_decision = D(preprocess(d_real_data))
            d_real_error = criterion(d_real_decision, Variable(torch.ones([1, 1])))
            d_real_error.backward()
            
            # Train D on fake
            
            d_gen_input = Variable(gi_sampler(minibatch_size, g_input_size))
            d_fake_data = G(d_gen_input).detach() 
            d_fake_decision = D(preprocess(d_fake_data.t()))
            d_fake_error = criterion(d_fake_decision, Variable(torch.zeros([1, 1])))
            d_fake_error.backward()
            d_optimizer.step()
            dre, dfe = extract(d_real_error)[0], extract(d_fake_error)[0]

        for g_index in range(g_steps):
            
            G.zero_grad()

            gen_input = Variable(gi_sampler(minibatch_size, g_input_size))
            g_fake_data = G(gen_input)
            dg_fake_decision = D(preprocess(g_fake_data.t()))
            g_error = criterion(dg_fake_decision, Variable(torch.ones([1,1])))  # Train G to pretend it's genuine

            g_error.backward()
            g_optimizer.step() 
            ge = extract(g_error)[0]

        if epoch % print_interval == 0:
            print("Epoch %s: D (%s real_err, %s fake_err) G (%s err); Real Dist (%s),  Fake Dist (%s) " %
                  (epoch, dre, dfe, ge, stats(extract(d_real_data)), stats(extract(d_fake_data))))

    

train()            
            

e:\pytorch\gan\lib\site-packages\torch\nn\functional.py:1594: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Epoch 0: D (0.471771776676178 real_err, 0.9878430962562561 fake_err) G (0.469330370426178 err); Real Dist ([3.977418804578483, 1.4935605372930503]),  Fake Dist ([-0.4849961636662483, 0.01139806261360074]) 
Epoch 100: D (0.5943524837493896 real_err, 0.6053906679153442 fake_err) G (0.7785900235176086 err); Real Dist ([3.9809389098286627, 1.4749677897229723]),  Fake Dist ([1.8211149561405182, 0.003892357562871782]) 
Epoch 200: D (0.1085495725274086 real_err, 0.10379469394683838 fake_err) G (2.3192827701568604 err); Real Dist ([3.926039694290608, 1.4870965543105505]),  Fake Dist ([4.298670175552368, 0.0111888263826689]) 
Epoch 300: D (0.7222245931625366 real_err, 0.6647629141807556 fake_err) G (0.704225480556488 err); Real Dist ([3.9167129578739406, 1.438908682653233]),  Fake Dist ([5.32268830871582, 1.8459378362409897]) 
Epoch 400: D (0.5354322195053101 real_err, 0.4953654408454895 fake_err) G (0.9113466739654541 err); Real Dist ([4.03533214572072, 1.5042457088158565]),  Fake Dist ([7.137

Epoch 4000: D (0.6954166293144226 real_err, 0.6922503113746643 fake_err) G (0.6933377385139465 err); Real Dist ([4.126245646744967, 1.4540631304737606]),  Fake Dist ([4.030944205760956, 1.4686045458556591]) 
Epoch 4100: D (0.6934343576431274 real_err, 0.6934424042701721 fake_err) G (0.6927588582038879 err); Real Dist ([4.107142077576369, 1.5126543337126677]),  Fake Dist ([3.960311860561371, 1.4636967280458217]) 
Epoch 4200: D (0.6906442046165466 real_err, 0.6930885314941406 fake_err) G (0.6926541328430176 err); Real Dist ([4.006866872310638, 1.3935900816532243]),  Fake Dist ([4.101149294376373, 1.4349195822567617]) 
Epoch 4300: D (0.693846583366394 real_err, 0.6930832862854004 fake_err) G (0.6933298707008362 err); Real Dist ([4.130034747540951, 1.5246937799392442]),  Fake Dist ([4.061487138986587, 1.4721154345730774]) 
Epoch 4400: D (0.6925657391548157 real_err, 0.6928547024726868 fake_err) G (0.6927366852760315 err); Real Dist ([3.9703128718733787, 1.4948838673714673]),  Fake Dist ([4

NameError: name 'matplotlib_is_available' is not defined